In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications import VGG16
from keras.optimizers import Adam

E:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.
Can not use cuDNN on context None: cannot compile with cuDNN. We got this error:
b'C:\\Users\\acer\\AppData\\Local\\Temp\\try_flags_ea8xokrs.c:4:19: fatal error: cudnn.h: No such file or directory\r\ncompilation terminated.\r\n'
Preallocating 1638/2048 Mb (0.800000) on cuda
Mapped name None to device cuda: GeForce 940M (0000:04:00.0)


In [2]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
input_shape = (3, img_width, img_height)
# Размер мини-выборки
batch_size = 64
# Количество изображений для обучения
nb_train_samples = 1646
# Количество изображений для проверки
nb_validation_samples = 540
# # Количество изображений для тестирования
# nb_test_samples = 1000

In [3]:
vgg16_net = VGG16(weights='imagenet', 
                  include_top=False, 
                  input_shape=(3, 150, 150))

In [4]:
vgg16_net.trainable = False

In [5]:
model = Sequential()
model.add(vgg16_net)
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512, 4, 4)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 771       
_________________________________________________________________
activation_2 (Activation)    (None, 3)                 0         
Total para

In [7]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=1e-5), 
              metrics=['accuracy'])

In [8]:
datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

# test_generator = datagen.flow_from_directory(
#     test_dir,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='categorical')

Found 1646 images belonging to 3 classes.
Found 540 images belonging to 3 classes.


In [9]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

Epoch 1/10
25/25 [==============================] - 146s 6s/step - loss: 1.0916 - acc: 0.4846 - val_loss: 0.7609 - val_acc: 0.7383
Epoch 2/10
25/25 [==============================] - 109s 4s/step - loss: 0.8434 - acc: 0.6383 - val_loss: 0.6197 - val_acc: 0.8086
Epoch 3/10
25/25 [==============================] - 100s 4s/step - loss: 0.7147 - acc: 0.7259 - val_loss: 0.5167 - val_acc: 0.8398
Epoch 4/10
25/25 [==============================] - 100s 4s/step - loss: 0.5942 - acc: 0.7738 - val_loss: 0.4459 - val_acc: 0.8555
Epoch 5/10
25/25 [==============================] - 101s 4s/step - loss: 0.4970 - acc: 0.8325 - val_loss: 0.3895 - val_acc: 0.8711
Epoch 6/10
25/25 [==============================] - 103s 4s/step - loss: 0.4397 - acc: 0.8496 - val_loss: 0.3452 - val_acc: 0.8848
Epoch 7/10
25/25 [==============================] - 108s 4s/step - loss: 0.3880 - acc: 0.8594 - val_loss: 0.3158 - val_acc: 0.8984
Epoch 8/10
25/25 [==============================] - 100s 4s/step - loss: 0.3512 - a

In [10]:
# scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
# print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

In [15]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("mnist_modelv10.json", "w")
json_file.write(model_json)
json_file.close()

In [16]:
model.save_weights("mnist_modelv10.h5")

In [71]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

img = image.load_img('1.jpg', target_size=(150, 150))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
preds = model.predict(x)
print(preds)
if preds[0][0]>=0.5 and preds[0][0]<=1.5:
    print('Машина')
if preds[0][1]>=0.5 and preds[0][1]<=1.5:
    print('Грузовик')
if preds[0][2]>=0.5 and preds[0][2]<=1.5:
    print('Мотоцикл')
if (preds[0][0]<0.5 or preds[0][0]>1.5) and (preds[0][1]<0.5 or preds[0][1]>1.5) and (preds[0][2]<0.5 or preds[0][2]>1.5):
    print('Ничего')


[[5.701723e-15 1.000000e+00 8.136865e-30]]
Грузовик
